In [59]:
from script.scraper_functions import *
from script.soupifier import *
from serde.json import to_json
import numpy as np
from itertools import chain
from yarl import URL

**TODO**:

- `ruff` $\Rightarrow$ linter, à utiliser plus dans le cadre du dvpt
- `pytest` avec subprocess.run pour tester le code
- `pytest-coverage`
- `poetry` avec gestion des dépendances
- `black` pour automatiquement indenter, etc.
- `doctest` pour tester les exemples des docstrings
- `bandit` pour gérer les vulnérabilités

***

- `yarl` pour travailler avec des URL

## Le code ci-dessous lance le `scraper`

- Tu peux changer le nombre d'*URL* de **19** à 1 ou 2 pour tester ça prendra beaucoup moins longtemps à générer
- Lien [`Darty`](https://www.darty.com/nav/extra/list?seller=311289-0-327183-311290-311291-328771-311452&cat=89054&state=new_offer&persist_sold_by_darty=true) $\Rightarrow$ en choisissant les produits uniquement vendus par Darty + Neufs on a 567 produits.
- Avoir un comparatif Boulanger/Darty pourrait être intéressant
- Voir pour récupérer les images des téléphones pour plus tard dans l'appli - voir : https://discuss.streamlit.io/t/add-image-and-header-to-streamlit-dataframe-table/36065/3

In [60]:
URL()

URL('')

In [61]:
session = create_session()
url_list = create_url_list(URL, 1)
pages = read_pages(url_list=url_list, session=session)

In [ ]:
all_urls = extract_all_urls(pages)
print("--------------------")
product_pages = extract_all_pages(all_urls, session)

Elapsed time for extract_all_urls function: 0.488 seconds.

--------------------
Product 1 -> Successfull Extraction.
Product 2 -> Successfull Extraction.
Product 3 -> Successfull Extraction.
Product 4 -> Successfull Extraction.
Product 5 -> Successfull Extraction.
Product 6 -> Successfull Extraction.
Product 7 -> Successfull Extraction.
Product 8 -> Successfull Extraction.
Product 9 -> Successfull Extraction.
Product 10 -> Successfull Extraction.
Product 11 -> Successfull Extraction.
Product 12 -> Successfull Extraction.
Product 13 -> Successfull Extraction.
Product 14 -> Successfull Extraction.
Product 15 -> Successfull Extraction.
Product 16 -> Successfull Extraction.
Product 17 -> Successfull Extraction.
Product 18 -> Successfull Extraction.
Product 19 -> Successfull Extraction.
Product 20 -> Successfull Extraction.
Product 21 -> Successfull Extraction.
Product 22 -> Successfull Extraction.
Product 23 -> Successfull Extraction.
Product 24 -> Successfull Extraction.
Product 25 -> Successfull Extraction.
Product 26 -> Successfull Extraction.


Elapsed time for extract_all_pages function: 59.688 seconds.

Il faudrait créer une fonction pour la génération de la liste des smartphones + l'export en `json`

In [ ]:
smartphone_list = list()
for article in product_pages:
    soup = BeautifulSoup(article, "html.parser")
    dict_features = extract_features(soup)
    smartphone = smartphone_characteristics(dict_features)
    smartphone_list.append(smartphone)

In [ ]:
smartphones_json = to_json(smartphone_list)

In [ ]:
file_path = "data/smartphones.json"

with open(file_path, "w", encoding="utf-8") as json_file:
    json_file.write(smartphones_json)

## Charge le Dataframe dans `polars`

In [2]:
import polars as pl
import polars.selectors as cs
from script.polars_gazprom import *

In [3]:
df_brut = pl.read_json("data/smartphones.json")

In [4]:
df_brut.select(pl.all().is_null().sum())

cpu,usb_type_c,model,repairability_index,brand,das_limbs,charging_type,das_chest,height,storage,stars,upgrade_storage,sim_number,reviews,ram,warranty,das_head,screen_tech,sensor,sensor_resolution,thickness,os,battery,price,screen_type,net_weight,made_in,fast_charging,screen_resolution,bluetooth,color,sim_type,width,screen_size,network,refresh_rate,cpu_details
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
5,86,0,16,0,13,17,13,3,0,0,1,5,0,29,92,13,2,81,1,3,2,24,0,4,18,48,2,0,0,1,0,3,0,0,337,210


### Pipeline

Sur `535` extractions, il y a `487` téléphones valides.

Pour obtenir la *PPI* (Pixels per Inch) on fait : 

$$PPI = \dfrac{\text{diagonal\_pixels}}{\text{screen\_size} (inch)}$$

In [5]:
df = NordStream(df_brut)

In [6]:
df

cpu,usb_type_c,model,repairability_index,brand,das_limbs,das_chest,height,storage,stars,upgrade_storage,reviews,ram,das_head,screen_tech,sensor,cam_1,cam_2,cam_3,thickness,battery,price,screen_type,net_weight,made_in,fast_charging,resolution_1,resolution_2,color,width,screen_size,network,diagonal_pixels,logprice,induction,ppi,mpx_backward_cam,random_col
str,bool,str,f64,str,f64,f64,f64,i64,f64,bool,i64,i64,f64,str,str,f64,f64,f64,f64,i64,f64,str,f64,str,bool,i64,i64,str,f64,f64,str,f64,f64,bool,f64,f64,i64
"""Qualcomm Snapd…",true,"""SONY Xperia 5 …",8.5,"""SONY""",2.11,1.05,15.4,128,0.0,true,0,8,0.95,"""OLED FHD+""","""3""",12.0,12.0,12.0,0.9,5000,999.0,"""Plat""",0.0,"""Thaïlande""",true,2520,1080,"""Argent""",0.7,6.1,"""5G""",2741.678318,6.906755,true,449.455462,36.0,731
"""Puce A17 Pro""",false,"""iPhone 15 Pro""",7.6,"""APPLE""",2.99,0.96,146.6,512,5.0,false,1,8,0.98,"""Super Retina X…","""3""",48.0,12.0,12.0,8.2,3650,1609.0,"""Plat""",187.0,"""Chine""",true,2556,1179,"""Bleu""",70.6,6.1,"""5G""",2814.813848,7.383368,true,461.444893,72.0,-499
"""Exynos 2200""",true,"""Samsung Galaxy…",8.2,"""SAMSUNG""",2.71,1.59,146.0,256,4.0,false,1,8,1.21,"""Dynamic AMOLED…","""3""",50.0,12.0,10.0,0.8,3700,857.59,"""Plat""",167.0,"""Viêt Nam""",true,2340,1080,"""Noir""",87.0,6.1,"""5G""",2577.207791,6.754126,true,422.493081,72.0,-45
"""8 coeurs jusqu…",true,"""Motorola G53""",7.1,"""MOTOROLA""",2.98,1.32,162.7,128,4.3333,true,3,4,0.98,"""LCD""","""2""",50.0,2.0,0.0,8.2,5000,199.0,"""Plat""",183.0,"""Chine""",false,1600,720,"""Bleu""",74.7,6.5,"""5G""",1754.536976,5.293305,false,269.928766,52.0,745
"""Google Tensor …",true,"""Google Pixel 8…",8.2,"""GOOGLE""",2.49,1.2,15.1,128,0.0,false,0,8,0.98,"""Actua ou OLED …","""3""",50.0,12.0,10.0,0.9,4500,799.0,"""Plat""",null,"""Viêt Nam""",true,1080,2400,"""Noir""",7.1,6.2,"""5G""",2631.805464,6.683361,true,424.484752,72.0,644
"""Puce A16 Bioni…",true,"""iPhone 15 Plus…",7.5,"""APPLE""",2.98,0.98,160.9,512,0.0,false,0,6,0.98,"""OLED""","""2""",48.0,12.0,0.0,7.8,4912,1499.0,"""Plat""",201.0,"""Chine""",true,2796,1290,"""Bleu""",77.8,6.7,"""5G""",3079.239517,7.312553,true,459.587988,60.0,-328
"""8 coeurs jusqu…",true,"""Xiaomi Redmi 9…",7.0,"""XIAOMI""",2.526,1.186,164.9,32,4.0,true,2,2,0.398,"""HD+""","""1""",13.0,0.0,0.0,9.0,5000,95.9,"""Plat""",194.0,"""Chine""",true,1600,720,"""Gris""",77.1,6.5,"""4G""",1754.536976,4.563306,false,269.928766,13.0,-24
"""Qualcomm Snapd…",true,"""HONOR Magic5 L…",8.0,"""HONOR""",2.81,1.27,161.6,256,4.1333,false,15,8,0.82,"""AMOLED (rigide…","""3""",64.0,5.0,2.0,7.9,5100,299.0,"""Bords incurvés…",175.0,"""Chine""",true,1080,2400,"""Noir""",73.9,6.7,"""5G""",2631.805464,5.700444,false,392.806786,71.0,-98
"""Google Tensor""",true,"""Google Pixel 6…",6.4,"""GOOGLE""",2.96,1.4,163.9,128,4.087,false,23,12,0.99,"""Non communiqué…","""3""",50.0,48.0,12.0,8.9,5000,633.67,"""Plat""",210.0,"""Chine""",true,3120,1440,"""Noir""",75.9,6.7,"""5G""",3436.277055,6.451528,true,512.877172,110.0,-739


In [50]:
#df.select(cs.by_dtype(pl.NUMERIC_DTYPES)).corr()

# Matrice de corrélation

In [7]:
import statsmodels.formula.api as smf
import statsmodels.formula as sm
from stargazer.stargazer import Stargazer
import numpy as np
from dataclasses import dataclass
from script.hedonic_regression import *

In [9]:
y_1 = "price"
y_2 = "logprice"
X = [
    "storage",
    "induction",
    "upgrade_storage",
    "ram",
    "screen_type",
    "network",
    "screen_size",
    "brand",
    "made_in",
    "das_limbs",
    "battery",
    "ppi",
    "mpx_backward_cam",
]

+ revoir le polars pour ajouter `total_pixels` aussi => multiplication de 1920x1080 (par exemple)

In [10]:
f_1 = formula(y_1, X)
f_2 = formula(y_2, X)
res_1 = execute_ols(df, f_1)
res_2 = execute_ols(df, f_2)

In [11]:
from pathlib import Path
root = Path(".").resolve()
data_folder = root / "data"
path_file = data_folder / "df_clean.csv"

On ajoute le prix prédit par la régression 1`E`1 à notre dataframe

In [12]:
df = df.with_columns(pl.from_numpy(res_1.predict(), schema=["predicted_price"]))

In [60]:
#df.select("price", "predicted_price", "model").filter(pl.col("predicted_price") < 0)

price,predicted_price,model
f64,f64,str
99.0,-35.420331,"""Motorola E13"""
106.59,-66.638032,"""Xiaomi Redmi A…"
99.0,-4.545654,"""XIAOMI Redmi 9…"
99.0,-126.592071,"""Motorola E20"""
114.79,-24.072381,"""XIAOMI Redmi 1…"
131.36,-132.537441,"""Motorola E22i…"
99.99,-4.545654,"""XIAOMI Redmi 9…"


In [13]:
df.write_csv(path_file, separator=";")

In [14]:
Stargazer([res_1, res_2])

Variables qui n'apportent rien dans la loglin-reg : 

- color
- net_weight
- model
- das_chest
- das_head

In [13]:
def feature_importance(df: pl.DataFrame, y:str):
    column_name = list()
    r2_list = list()
    for column in df.columns:
        ols_model = smf.ols(
        formula=f"{y}~{column}",
        data=df)
        res = ols_model.fit()
        column_name.append(column)
        r2_list.append(res.rsquared)

    feature_imp_df = pl.DataFrame({'Column': column_name, 'R-squared': r2_list}).sort("R-squared", descending=True)
    return feature_imp_df

In [14]:
df_features = feature_importance(df, "price")

In [15]:
import plotly.express as px

In [16]:
fig = px.bar(df_features, x='Column', y='R-squared', title='Feature Importance (R-squared)')
fig.update_xaxes(title_text='Feature')
fig.update_yaxes(title_text='R-squared')
fig.show()

In [ ]:
# res2 = res.get_robustcov_results(cov_type='HC1')
# robuste std errors

- fuzzy match à mettre en place ici (pour la technologie de l'écran) ?
- ajouter une feature complètement random (pour la partie Machine Learning) puis regarder l'importance des variables : si les variables sont moins importantes que la random, alors on les dégage

In [ ]:
# df.select("cpu").unique().to_numpy()
# pour le cpu on va préférer retravailler pour avoir la fréquence en GHz plutot